# Reading Progress Module Notebook

In [2]:
%run /OEA_py

StatementMeta(, 83, -1, Finished, Available)

2022-12-20 18:43:34,835 - OEA - DEBUG - OEA initialized.
2022-12-20 18:43:34,835 - OEA - DEBUG - OEA initialized.
OEA initialized.


In [3]:
# 0) Initialize the OEA framework.
oea = OEA()

StatementMeta(spark3p1sm, 83, 3, Finished, Available)

2022-12-20 18:43:35,196 - OEA - DEBUG - OEA initialized.
2022-12-20 18:43:35,196 - OEA - DEBUG - OEA initialized.
2022-12-20 18:43:35,196 - OEA - DEBUG - OEA initialized.
OEA initialized.


## Data Loading and Subsetting

In [4]:
#Loading the data
from pyspark.sql.types import IntegerType

dfInsights_techActivity = oea.load('M365', 'TechActivity_pseudo')

dfInsights_aaduserpersonmapping = oea.load('M365', 'AadUserPersonMapping_pseudo')
dfInsights_person = oea.load('M365', 'Person_pseudo')
dfInsights_personOrgRole = oea.load('M365', 'PersonOrganizationRole_pseudo')
dfInsights_organization = oea.load('M365', 'Organization_pseudo')
dfInsights_refDefinition = oea.load('M365', 'RefDefinition_pseudo')

StatementMeta(spark3p1sm, 83, 4, Finished, Available)

In [5]:
#Subsetting and joining the data
dfInsights = dfInsights_personOrgRole.join(dfInsights_person, dfInsights_personOrgRole.PersonId_pseudonym == dfInsights_person.Id_pseudonym, how='inner')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId')

dfInsights = dfInsights.join(dfInsights_organization, dfInsights.OrganizationId == dfInsights_organization.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Name', 'OrganizationName')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'RefRoleId', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')

dfInsights = dfInsights.join(dfInsights_refDefinition, dfInsights.RefRoleId == dfInsights_refDefinition.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Code', 'PersonRole')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'RefGradeLevelId', 'OrganizationId', 'OrganizationName')

dfInsights = dfInsights.join(dfInsights_refDefinition, dfInsights.RefGradeLevelId == dfInsights_refDefinition.Id, how='inner')
dfInsights = dfInsights.withColumnRenamed('Code', 'StudentGrade')
dfInsights = dfInsights.select('PersonId_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade', 'OrganizationId', 'OrganizationName')

dfInsights_aaduserpersonmapping = dfInsights_aaduserpersonmapping.withColumnRenamed('PersonId_pseudonym', 'StudentId_internal_pseudonym')
dfInsights = dfInsights.join(dfInsights_aaduserpersonmapping, dfInsights.PersonId_pseudonym == dfInsights_aaduserpersonmapping.StudentId_internal_pseudonym, how='inner')
dfInsights = dfInsights.withColumnRenamed('ObjectId_pseudonym', 'StudentId_external_pseudonym')
dfInsights = dfInsights.select('StudentId_internal_pseudonym', 'StudentId_external_pseudonym', 'Surname', 'GivenName', 'MiddleName', 'PersonRole', 'StudentGrade', 'OrganizationId', 'OrganizationName')
display(dfInsights.limit(5))

StatementMeta(spark3p1sm, 83, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 32974c31-246a-484c-b306-bab8001953ac)

In [6]:
dfReadingProgress = dfInsights_techActivity.where("AppName == 'ReadingProgress'")
dfReadingProgress = dfReadingProgress.select('ActorId_pseudonym', 'SignalType', 'StartTime', 'AppName', 'Action', 'ReadingSubmissionWordsPerMinute', 'ReadingSubmissionAccuracyScore', 'ReadingSubmissionRepetitionsCount', 'ReadingSubmissionInsertionsCount', 'ReadingSubmissionMispronunciationCount', 'ReadingSubmissionObmissionCount', 'ReadingSubmissionAttemptNumber', 'ReadingAssignmentWordCount', 'ReadingAssignmentFleschKincaidGradeLevel', 'ReadingAssignmentLanguage')
dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionAccuracyScore', dfReadingProgress['ReadingSubmissionAccuracyScore'].cast(IntegerType()))
display(dfReadingProgress.limit(5))

StatementMeta(spark3p1sm, 83, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 37e04d0c-e95f-4125-acc1-ffc264c1f618)

## Data Aggregations

In [7]:
from pyspark.sql import functions as F
dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionRepetitionsRate', F.col('ReadingSubmissionRepetitionsCount')/F.col('ReadingAssignmentWordCount') * 100) 
dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionMispronunciationRate', F.col('ReadingSubmissionMispronunciationCount')/F.col('ReadingAssignmentWordCount') * 100) 
dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionInsertionsRate', F.col('ReadingSubmissionInsertionsCount')/F.col('ReadingAssignmentWordCount') * 100) 
dfReadingProgress = dfReadingProgress.withColumn('ReadingSubmissionObmissionRate', F.col('ReadingSubmissionObmissionCount')/F.col('ReadingAssignmentWordCount') * 100) 
display(dfReadingProgress.limit(5))

StatementMeta(spark3p1sm, 83, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, c5d08973-19fb-4adf-b6b7-01b8ba862225)

## Write to Stage 2p

In [8]:
dfReadingProgress.coalesce(1).write.format('delta').mode('overwrite').option('header', True).save(oea.stage2p + '/reading_progress/ReadingProgress_pseudo')
dfInsights.coalesce(1).write.format('delta').mode('overwrite').option('header', True).save(oea.stage2p + '/reading_progress/Student_pseudo')


StatementMeta(spark3p1sm, 83, 8, Finished, Available)